#   **LangChain Custom Tool** (Part2)

- LangChain과 외부 API 통합

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **외부 API 연동** 

- LangChain에서는 **사용자 정의 도구**를 통해 외부 API와의 연동이 가능

- **Tool** 클래스를 상속받아 필요한 기능을 구현하며, `name`과 `description`을 필수로 정의

- 도구는 단일 기능을 수행하는 **함수 형태**로 구현되며, 입력과 출력이 명확하게 설정함

`(1) Yahoo Finance API` 

In [3]:
#  yfinance 설치 : pip install yfinance 또는 poetry add yfinance
import yfinance as yf

dat = yf.Ticker("MSFT")

In [4]:
# 기업정보
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security and Compliance, and Copilot; Microsoft 365 Commercial products, such as Windows Commercial on-premises and Office licensed services; Microsoft 365 Consumer products and cloud services, such as Microsoft 365 Consumer subscriptions, Office licensed on-premises, and other consu

In [5]:
# 주요 일정
dat.calendar

{'Dividend Date': datetime.date(2025, 12, 11),
 'Ex-Dividend Date': datetime.date(2025, 11, 20),
 'Earnings Date': [datetime.date(2025, 10, 30)],
 'Earnings High': 3.79021,
 'Earnings Low': 3.5,
 'Earnings Average': 3.663,
 'Revenue High': 76641448730,
 'Revenue Low': 70066000000,
 'Revenue Average': 75374737710}

In [6]:
# 2022년 1월 3일 ~ 4일의 데이터를 판다스 데이터프레임으로 출력 
result = dat.history(start="2022-01-03", end="2022-01-05") 
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,325.086220,327.655108,319.686689,324.504578,28865100,0.0,0.0
2022-01-04 00:00:00-05:00,324.582127,324.940827,316.138715,318.940277,32674300,0.0,0.0


In [7]:
# 인덱스 초기화 
result = result.reset_index()
result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03 00:00:00-05:00,325.086220,327.655108,319.686689,324.504578,28865100,0.0,0.0
1,2022-01-04 00:00:00-05:00,324.582127,324.940827,316.138715,318.940277,32674300,0.0,0.0


In [8]:
# 날짜 부분만 추출
result['Date'] = result['Date'].dt.strftime('%Y-%m-%d')

result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03,325.086220,327.655108,319.686689,324.504578,28865100,0.0,0.0
1,2022-01-04,324.582127,324.940827,316.138715,318.940277,32674300,0.0,0.0


In [9]:
# 데이터프레임을 딕셔너리로 변환
result_dict = result.to_dict(orient='records') 
result_dict

[{'Date': '2022-01-03',
  'Open': 325.0862198389616,
  'High': 327.6551075166869,
  'Low': 319.68668922154853,
  'Close': 324.50457763671875,
  'Volume': 28865100,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2022-01-04',
  'Open': 324.5821267708263,
  'High': 324.94082734218637,
  'Low': 316.1387146989663,
  'Close': 318.9402770996094,
  'Volume': 32674300,
  'Dividends': 0.0,
  'Stock Splits': 0.0}]

`(2) 데이터 연동 및 출력 포맷` 

In [10]:
# 외부 API 연동하는 함수 (yfinance 사용)

from langchain_core.tools import ToolException
from typing import Dict, Optional
from datetime import datetime, timedelta
import yfinance as yf

def get_stock_price(symbol: str, date: Optional[str] = None) -> Dict:
    """yfiance 사용하여 특정 날짜의 주식의 가격 정보를 조회합니다."""

    if date and not is_valid_date(date):
        raise ToolException(f"잘못된 날짜 형식입니다: {date}")
    
    try:
        stock = yf.Ticker(symbol)
        # 특정 날짜의 주식 가격 정보 조회 
        if date:
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 해날 날짜로부터 과거 5일간의 주식 가격 정보 조회
            if price.empty:
                end = start - timedelta(days=5)
                price = stock.history(start=end, end=start)

        # 특정 날짜가 없으면 최근 5일간의 주식 가격 정보 조회
        else:
            price = stock.history(period="5d")
            
        # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
        df = price.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df.to_dict(orient='records')[-1]
    
    except Exception as e:
        raise ToolException(str(e))
    

def is_valid_date(date_str: str) -> bool:
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    

# 함수 실행
result = get_stock_price("AAPL")

# 결과 출력
print(result)

{'Date': '2025-10-22', 'Open': 262.6499938964844, 'High': 262.8500061035156, 'Low': 255.42999267578125, 'Close': 258.45001220703125, 'Volume': 44954300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [11]:
# 함수 실행 (날짜 지정)
result = get_stock_price("AAPL", "2025-01-03")
print(result)

{'Date': '2025-01-03', 'Open': 242.49916076660156, 'High': 243.31625226562878, 'Low': 241.03435939639962, 'Close': 242.49916076660156, 'Volume': 40244100, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(3) StructuredTool 도구 변환` 

In [12]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool_basic = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    description="yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.",
)

# 도구 실행 (정상)
result = stock_tool_basic.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-10-22', 'Open': 262.6499938964844, 'High': 262.8500061035156, 'Low': 255.42999267578125, 'Close': 258.45001220703125, 'Volume': 44954300, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(4) LLM 사용하여 도구 사용` 

In [13]:
from langchain_openai import ChatOpenAI

# OpenAI GPT-4.1-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 바인딩
llm_with_tool = llm.bind_tools([stock_tool_basic])

# 도구 호출
result = llm_with_tool.invoke("애플의 주식 가격을 알려줘")
pprint(result.tool_calls)
print("-"*100)

# ToolCall을 도구에 전달하여 결과 확인
tool_call = result.tool_calls[0]
result = stock_tool_basic.invoke(tool_call)

print(result)

[{'args': {'symbol': 'AAPL'},
  'id': 'call_IZK5bJLyKnhG5VhsDzLEeWGM',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
----------------------------------------------------------------------------------------------------
content='{"Date": "2025-10-22", "Open": 262.6499938964844, "High": 262.8500061035156, "Low": 255.42999267578125, "Close": 258.45001220703125, "Volume": 44954300, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_IZK5bJLyKnhG5VhsDzLEeWGM'


In [14]:
# 도구 호출 (날짜 지정)
result = llm_with_tool.invoke("애플의 2025년 1월 3일 주식 가격을 알려줘")
pprint(result.tool_calls)

# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    result = stock_tool_basic.invoke(tool_call)
    print(result)

[{'args': {'date': '2025-01-03', 'symbol': 'AAPL'},
  'id': 'call_naJ9szILss7cWOQhBEXc6gDD',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
content='{"Date": "2025-01-03", "Open": 242.49916076660156, "High": 243.31625226562878, "Low": 241.03435939639962, "Close": 242.49916076660156, "Volume": 40244100, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_naJ9szILss7cWOQhBEXc6gDD'


In [15]:
from langchain.agents import create_agent

# 도구 실행 에이전트 생성 
stock_agent = create_agent(
    model=llm,
    tools=[stock_tool_basic],
    system_prompt="당신은 주식 정보를 제공하는 AI 어시스턴트입니다."
)

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [{"role": "user", "content": "애플의 2025년 1월 3일 주식 가격을 알려줘"}]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='f7679d90-b33d-42eb-bdac-11755880029f'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 100, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CTouZMmLSX8FmxCzSUhvP7Zrbsfwa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e318f3d6-5d7b-4a2b-97ce-c9ca366db136-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': 'AAPL', 'date': '2025-01-03'}, 'id': 'call_PFOlwTtHVSsZgs0BPuwiEhQJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens

In [16]:
# 도구 실행 에이전트 생성 (return_direct=True)
stock_tool_basic.return_direct = True
stock_agent = create_agent(
    model=llm,
    tools=[stock_tool_basic],
    system_prompt="당신은 주식 정보를 제공하는 AI 어시스턴트입니다."
)

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [{"role": "user", "content": "애플의 2025년 1월 3일 주식 가격을 알려줘"}]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='849e6f2f-d459-4ed0-bd80-8fcc2c783090'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 100, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CToucjDiVcMJfFyXEeA1GTYk5K8z7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--323cdd97-2ed8-402f-99e1-4467dbfaddd3-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': 'AAPL', 'date': '2025-01-03'}, 'id': 'call_fVEUsRdu8qexS4MzezdBPgUS', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens

---

### [실습]

- yfinance 사용하여 특정 기간(시작일 ~ 종료일) 동안의 거래 데이터를 가져오는 도구를 정의 

- **Hint**: history 메소드의 start, end 속성에 날짜 지정 

In [17]:
from typing import List, Dict
import yfinance as yf
from datetime import datetime

# 1단계: 일반 함수로 정의
def get_stock_period_data(symbol: str, start_date: str, end_date: str) -> List[Dict]:
    """yfinance를 사용하여 특정 기간(시작일 ~ 종료일) 동안의 주식 거래 데이터를 가져옵니다.
    
    Args:
        symbol (str): 주식 심볼 (예: AAPL, TSLA)
        start_date (str): 시작 날짜 (YYYY-MM-DD 형식)
        end_date (str): 종료 날짜 (YYYY-MM-DD 형식)
    
    Returns:
        List[Dict]: 기간 동안의 주식 거래 데이터 리스트
    """
    stock = yf.Ticker(symbol)
    
    # 특정 기간의 주식 가격 정보 조회 (start, end 속성 사용)
    price = stock.history(start=start_date, end=end_date)
    
    # 데이터프레임을 딕셔너리로 변환하여 반환
    df = price.reset_index()
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    return df.to_dict(orient='records')

# 함수 실행 예시 (일반 함수로 호출)
result = get_stock_period_data("AAPL", "2025-01-01", "2025-01-10")
print(result)

# 2단계: 함수를 도구로 변환
from langchain_core.tools import tool

@tool
def get_stock_period_data_tool(symbol: str, start_date: str, end_date: str) -> List[Dict]:
    """yfinance를 사용하여 특정 기간(시작일 ~ 종료일) 동안의 주식 거래 데이터를 가져옵니다.
    
    Args:
        symbol (str): 주식 심볼 (예: AAPL, TSLA)
        start_date (str): 시작 날짜 (YYYY-MM-DD 형식)
        end_date (str): 종료 날짜 (YYYY-MM-DD 형식)
    
    Returns:
        List[Dict]: 기간 동안의 주식 거래 데이터 리스트
    """
    return get_stock_period_data(symbol, start_date, end_date)

# 도구로 호출 (invoke 메서드 사용)
result_tool = get_stock_period_data_tool.invoke({"symbol": "AAPL", "start_date": "2025-01-01", "end_date": "2025-01-10"})
print("\n도구로 호출한 결과:")
print(result_tool)

[{'Date': '2025-01-02', 'Open': 248.04942813526918, 'High': 248.21884015778812, 'Low': 240.96459363401945, 'Close': 242.98741149902344, 'Volume': 55740700, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Date': '2025-01-03', 'Open': 242.49916076660156, 'High': 243.31625226562878, 'Low': 241.03435939639962, 'Close': 242.49916076660156, 'Volume': 40244100, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Date': '2025-01-06', 'Open': 243.4457849039297, 'High': 246.45510633108967, 'Low': 242.33971076643533, 'Close': 244.1333465576172, 'Volume': 45045600, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Date': '2025-01-07', 'Open': 242.12049100282684, 'High': 244.681407317594, 'Low': 240.496267218822, 'Close': 241.3532257080078, 'Volume': 40856000, 'Dividends': 0.0, 'Stock Splits': 0.0}, {'Date': '2025-01-08', 'Open': 241.0642520177986, 'High': 242.8479287466886, 'Low': 239.2008716539854, 'Close': 241.84149169921875, 'Volume': 37628900, 'Dividends': 0.0, 'Stock Splits': 0.0}]

도구로 호출한 결과:
[{'Date': '2025-01-02

---

### 2. **도구 에러 처리** 

> **⚠️ LangChain v1.0 업데이트**  
> - 도구 에러 처리는 `handle_tool_error` 파라미터로 계속 사용 가능
> - 추가로 **미들웨어(Middleware)** 를 통한 고급 에러 처리도 지원
> - 미들웨어는 더 유연한 에러 처리와 재시도 로직 구현 가능

- **handle_tool_error** 매개변수를 통해 도구의 오류를 체계적으로 관리

- 에이전트와 도구 사이의 **에러 처리 로직**을 명확하게 정의할 수 있음

- LangChain이 제공하는 **기본 에러 처리 방식**으로 안정적인 실행 보장

- 도구 실행 중 발생하는 **예외 상황**을 효과적으로 제어 가능


In [18]:
# 함수 실행 (날짜 오류)
result = get_stock_price("AAPL", "2022-01-01")
print(result)

$AAPL: possibly delisted; no price data found  (1d 2022-01-01 00:00:00 -> 2022-01-02 00:00:00)


{'Date': '2021-12-31', 'Open': 174.5999629626884, 'High': 175.71762176050464, 'Low': 173.7862266948401, 'Close': 174.0901641845703, 'Volume': 64062300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [19]:
# 도구 실행 (날짜 오류)
result = stock_tool_basic.invoke({"symbol": "AAPL", "date": "2022-01-01"})
print(result)

$AAPL: possibly delisted; no price data found  (1d 2022-01-01 00:00:00 -> 2022-01-02 00:00:00)


{'Date': '2021-12-31', 'Open': 174.5999629626884, 'High': 175.71762176050464, 'Low': 173.7862266948401, 'Close': 174.0901641845703, 'Volume': 64062300, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(1) 기본 에러 처리`
   - *handle_tool_error=True*를 사용
   - ToolException의 메시지를 그대로 반환

In [20]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool_basic = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    handle_tool_error=True  # 기본 에러 메시지 반환
)

# 도구 실행 (정상)
result = stock_tool_basic.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-10-22', 'Open': 262.6499938964844, 'High': 262.8500061035156, 'Low': 255.42999267578125, 'Close': 258.45001220703125, 'Volume': 44954300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [21]:
# 도구 실행 (날짜 오류)
result = stock_tool_basic.invoke({"symbol": "AAPL", "date": "22-01-01"})
print(result)

잘못된 날짜 형식입니다: 22-01-01


`(2) 커스텀 에러 메시지`
   - *handle_tool_error*="에러 메시지"를 사용
   - 모든 에러에 대해 동일한 메시지 반환

In [22]:
# StructuredTool로 도구 생성
stock_tool_custom = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_custom",
    handle_tool_error="유효한 날짜 형식(YYYY-MM-DD)이 아닙니다. 다시 입력해주세요."
)

# 도구 실행 (정상)
result = stock_tool_custom.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-10-22', 'Open': 262.6499938964844, 'High': 262.8500061035156, 'Low': 255.42999267578125, 'Close': 258.45001220703125, 'Volume': 44954300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [23]:
# 도구 실행 (날짜 오류)
result = stock_tool_custom.invoke({"symbol": "AAPL", "date": "22-02-02"})
print(result)

유효한 날짜 형식(YYYY-MM-DD)이 아닙니다. 다시 입력해주세요.


`(3) 커스텀 에러 처리 함수`
   - handle_tool_error에 함수 전달
   - 에러 타입에 따라 다른 처리 가능

In [24]:
# 커스텀 에러 처리 함수
def custom_error_handler(error: Exception) -> str:
    if "잘못된 날짜" in str(error):
        return f"날짜 형식 오류: {str(error)}. YYYY-MM-DD 형식으로 입력해주세요."
    else:
        return f"도구 실행 중 오류가 발생했습니다: {str(error)}"

# StructuredTool로 도구 생성 (커스텀 에러 처리 함수)
stock_tool_custom_fn = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_custom_fn",
    handle_tool_error=custom_error_handler
)

In [25]:
# 도구 실행 (정상)
result = stock_tool_custom_fn.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-10-22', 'Open': 262.6499938964844, 'High': 262.8500061035156, 'Low': 255.42999267578125, 'Close': 258.45001220703125, 'Volume': 44954300, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [26]:
# 도구 실행 (날짜 오류)
result = stock_tool_custom_fn.invoke({"symbol": "AAPL", "date": "22-01-01"})
print(result)

날짜 형식 오류: 잘못된 날짜 형식입니다: 22-01-01. YYYY-MM-DD 형식으로 입력해주세요.


`(4) 미들웨어를 통한 고급 에러 처리` 

- LangChain v1.0에서 추가된 기능

In [27]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

# 미들웨어 정의
@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 에러를 처리하는 미들웨어"""
    try:
        return handler(request)
    except Exception as e:
        # 커스텀 에러 메시지 반환
        return ToolMessage(
            content=f"🛡️ 미들웨어가 에러를 처리했습니다: {str(e)}. 입력값을 확인해주세요.",
            tool_call_id=request.tool_call["id"]
        )

print("=" * 100)
print("테스트 1: 미들웨어 없이 실행 (에러 발생 시 중단)")
print("=" * 100)

# 미들웨어 없는 에이전트 생성
agent_without_middleware = create_agent(
    model=llm,
    tools=[stock_tool_basic],
    system_prompt="당신은 주식 정보를 제공하는 AI 어시스턴트입니다."
)

try:
    # 잘못된 날짜로 테스트 (에러 발생)
    result1 = agent_without_middleware.invoke(
        {"messages": [{"role": "user", "content": "애플의 202-01-01 주식 가격을 알려줘"}]}
    )
    print("✅ 실행 완료")
    for msg in result1['messages']:
        print(f"  {msg.type}: {msg.content[:80] if hasattr(msg, 'content') else msg}")
except Exception as e:
    print(f"❌ 에러로 실행 중단: {type(e).__name__}")
    print(f"   에러 내용: {str(e)[:150]}")

print("\n" + "=" * 100)
print("테스트 2: 미들웨어와 함께 실행 (에러를 안전하게 처리)")
print("=" * 100)

# 미들웨어를 적용한 에이전트 생성
agent_with_middleware = create_agent(
    model=llm,
    tools=[stock_tool_basic],
    middleware=[handle_tool_errors],
    system_prompt="당신은 주식 정보를 제공하는 AI 어시스턴트입니다."
)

try:
    # 동일한 잘못된 날짜로 테스트
    result2 = agent_with_middleware.invoke(
        {"messages": [{"role": "user", "content": "애플의 202-01-01 주식 가격을 알려줘"}]}
    )
    print("✅ 미들웨어가 에러를 처리하여 실행 완료")
    print("\n전체 메시지 목록:")
    pprint(result2['messages'])
except Exception as e:
    print(f"❌ 예상치 못한 에러: {type(e).__name__}: {str(e)[:150]}")

테스트 1: 미들웨어 없이 실행 (에러 발생 시 중단)
✅ 실행 완료
  human: 애플의 202-01-01 주식 가격을 알려줘
  ai: 
  tool: 잘못된 날짜 형식입니다: 202-01-01
  ai: 입력하신 날짜 형식이 잘못되었습니다. 정확한 날짜 형식(예: 2020-01-01)으로 다시 알려주시면 애플의 주식 가격을 조회해드리겠습니다.

테스트 2: 미들웨어와 함께 실행 (에러를 안전하게 처리)
✅ 미들웨어가 에러를 처리하여 실행 완료

전체 메시지 목록:
[HumanMessage(content='애플의 202-01-01 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='abfc2d8d-eb52-40f9-80a9-e46c3619106e'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 97, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CToukVCM8vCwBk1i9idAJBrZtQ3E6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': No

In [28]:
# Cell 44의 결과를 상세히 출력 (테스트 2의 결과)
print("=" * 100)
print("미들웨어 적용 결과 상세 분석")
print("=" * 100)

if 'result2' in locals():
    for i, msg in enumerate(result2['messages'], 1):
        print(f"\n[메시지 {i}] {msg.type}")
        print("-" * 80)
        if hasattr(msg, 'content') and msg.content:
            print(f"내용: {msg.content}")
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"도구 호출: {msg.tool_calls}")
        if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
            print(f"도구 호출 ID: {msg.tool_call_id}")
else:
    print("⚠️ Cell 44를 먼저 실행해주세요.")

미들웨어 적용 결과 상세 분석

[메시지 1] human
--------------------------------------------------------------------------------
내용: 애플의 202-01-01 주식 가격을 알려줘

[메시지 2] ai
--------------------------------------------------------------------------------
도구 호출: [{'name': 'stock_price_basic', 'args': {'symbol': 'AAPL', 'date': '202-01-01'}, 'id': 'call_VAfyuSWX9YCks7u9Zy4g2gqj', 'type': 'tool_call'}]

[메시지 3] tool
--------------------------------------------------------------------------------
내용: 잘못된 날짜 형식입니다: 202-01-01
도구 호출 ID: call_VAfyuSWX9YCks7u9Zy4g2gqj

[메시지 4] ai
--------------------------------------------------------------------------------
내용: 입력하신 날짜 형식이 잘못되었습니다. 정확한 날짜를 다시 알려주시겠어요? 예를 들어, 2020-01-01 같은 형식으로요.


---

### [실습]

- 도구 에러 처리 유형을 살펴 보고, 직접 수정하여 적용해봅니다. 

In [29]:
# 방법 1: 기본 에러 메시지 반환
stock_tool_basic_error = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic_error",
    handle_tool_error=True  # 기본 에러 메시지 반환
)

# 테스트 (날짜 오류)
print("=== 방법 1: 기본 에러 처리 ===")
result = stock_tool_basic_error.invoke({"symbol": "AAPL", "date": "22-01-01"})
print(result)
print()

# 방법 2: 커스텀 에러 메시지
stock_tool_custom_msg = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_custom_msg",
    handle_tool_error="⚠️ 날짜 형식이 올바르지 않습니다. YYYY-MM-DD 형식으로 입력해주세요."
)

# 테스트 (날짜 오류)
print("=== 방법 2: 커스텀 메시지 ===")
result = stock_tool_custom_msg.invoke({"symbol": "AAPL", "date": "2022/01/01"})
print(result)
print()

# 방법 3: 커스텀 에러 처리 함수
def advanced_error_handler(error: Exception) -> str:
    """고급 에러 처리 함수 - 에러 타입에 따라 다른 메시지 반환"""
    error_msg = str(error)
    
    if "잘못된 날짜" in error_msg:
        return f"📅 날짜 형식 오류: {error_msg}. YYYY-MM-DD 형식으로 입력해주세요."
    elif "찾을 수 없습니다" in error_msg:
        return f"🔍 데이터 조회 실패: {error_msg}. 주식 심볼을 확인해주세요."
    else:
        return f"❌ 도구 실행 중 오류가 발생했습니다: {error_msg}"

stock_tool_advanced = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_advanced",
    handle_tool_error=advanced_error_handler
)

# 테스트 (날짜 오류)
print("=== 방법 3: 고급 에러 처리 함수 ===")
result = stock_tool_advanced.invoke({"symbol": "AAPL", "date": "invalid-date"})
print(result)

=== 방법 1: 기본 에러 처리 ===
잘못된 날짜 형식입니다: 22-01-01

=== 방법 2: 커스텀 메시지 ===
⚠️ 날짜 형식이 올바르지 않습니다. YYYY-MM-DD 형식으로 입력해주세요.

=== 방법 3: 고급 에러 처리 함수 ===
📅 날짜 형식 오류: 잘못된 날짜 형식입니다: invalid-date. YYYY-MM-DD 형식으로 입력해주세요.


---

### 3. **BaseTool 상속** 

- **BaseTool**을 상속하여 더 복잡한 도구 구현 가능

- `_run` 메소드에서 **동기 실행 로직** 구현

- `_arun` 메소드에서 **비동기 실행 로직** 구현 (선택사항)

- Pydantic 모델로 **입력 스키마**를 명확하게 정의

`(1) 입력 스키마 정의`

In [30]:
from pydantic import BaseModel, Field

class StockPriceInput(BaseModel):
    """주식 가격 조회 도구의 입력 스키마"""
    symbol: str = Field(description="주식 심볼 (예: AAPL, TSLA)")
    date: str = Field(description="조회할 날짜 (YYYY-MM-DD 형식)")

`(2) BaseTool 상속하여 도구 구현`

In [31]:
from langchain_core.tools import BaseTool
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class StockPriceTool(BaseTool):
    name: str = "StockPrice"
    description: str = "yfinance를 사용하여 특정 날짜의 주식 가격 정보를 조회합니다."
    args_schema: type[BaseModel] = StockPriceInput
    return_direct: bool = False

    def _run(
        self,
        symbol: str,
        date: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> dict:
        """도구를 동기적으로 실행합니다."""
        
        # 날짜 유효성 검사
        if not is_valid_date(date):
            raise ToolException(f"잘못된 날짜 형식입니다: {date}")

        try:
            stock = yf.Ticker(symbol)
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 최근 거래일 데이터 조회
            if price.empty:
                price = stock.history(period="5d")
                if price.empty:
                    raise ToolException(f"{symbol}에 대한 주식 데이터를 찾을 수 없습니다.")
                
                price = price.reset_index()
                price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')
                result = price.to_dict(orient="records")[-1]
                
                return {
                    "symbol": symbol,
                    "date": result.get("Date"),
                    "open": result.get("Open"),
                    "high": result.get("High"),
                    "low": result.get("Low"),
                    "close": result.get("Close"),
                    "volume": result.get("Volume"),
                    "comment": f"{date} 날짜의 데이터가 없어 최근 거래일인 {result.get('Date')} 날짜의 주식 가격 정보 조회"
                }

            # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
            price = price.reset_index()
            price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')
            result = price.to_dict(orient="records")[-1]
            return {
                "symbol": symbol,
                "date": result.get("Date"),
                "open": result.get("Open"),
                "high": result.get("High"),
                "low": result.get("Low"),
                "close": result.get("Close"),
                "volume": result.get("Volume"),
                "comment": f"{result.get('Date')} 날짜의 주식 가격 정보 조회"
            }
                
        except Exception as e:
            raise ToolException(str(e))

    
    async def _arun(
        self,
        symbol: str,
        date: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> dict:
        """도구를 비동기적으로 실행합니다."""
        return self._run(symbol, date, run_manager.get_sync() if run_manager else None)

`(3) 도구 실행`

In [32]:
# 도구 생성
stock_tool = StockPriceTool()

# 도구 속성
print(stock_tool.name)
print(stock_tool.description)
print(stock_tool.args_schema)
print(stock_tool.return_direct)

StockPrice
yfinance를 사용하여 특정 날짜의 주식 가격 정보를 조회합니다.
<class '__main__.StockPriceInput'>
False


In [33]:
# 도구 실행 (거래일이 없는 날짜)
result = stock_tool.invoke({"symbol": "TSLA", "date": "2023-12-25"})
pprint(result)


$TSLA: possibly delisted; no price data found  (1d 2023-12-25 00:00:00 -> 2023-12-26 00:00:00)


{'close': 438.9700012207031,
 'comment': '2023-12-25 날짜의 데이터가 없어 최근 거래일인 2025-10-22 날짜의 주식 가격 정보 조회',
 'date': '2025-10-22',
 'high': 445.5400085449219,
 'low': 429.0,
 'open': 443.45001220703125,
 'symbol': 'TSLA',
 'volume': 81274900}


In [34]:
# 도구 실행 (거래일이 있는 날짜)
result = stock_tool.invoke({"symbol": "AAPL", "date": "2022-01-03"})
pprint(result)


{'close': 178.44313049316406,
 'comment': '2022-01-03 날짜의 주식 가격 정보 조회',
 'date': '2022-01-03',
 'high': 179.2960912081591,
 'low': 174.22740989529922,
 'open': 174.34505345884557,
 'symbol': 'AAPL',
 'volume': 104487900}


In [35]:
# LLM에 도구 바인딩하여 사용
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tool = llm.bind_tools([stock_tool])

# 도구 호출
result = llm_with_tool.invoke("애플의 2022년 1월 3일 주식 가격을 알려줘")
pprint(result.tool_calls)


[{'args': {'date': '2022-01-03', 'symbol': 'AAPL'},
  'id': 'call_2y7LW4SUrbgVwR00LeoHv7AE',
  'name': 'StockPrice',
  'type': 'tool_call'}]


In [36]:
# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    msg = stock_tool.invoke(tool_call)
    pprint(msg.content)


('{"symbol": "AAPL", "date": "2022-01-03", "open": 174.34505345884557, "high": '
 '179.2960912081591, "low": 174.22740989529922, "close": 178.44313049316406, '
 '"volume": 104487900, "comment": "2022-01-03 날짜의 주식 가격 정보 조회"}')


In [37]:
# 도구 실행 (거래일이 없는 날짜)
result = llm_with_tool.invoke("테슬라의 2023년 12월 25일 주식 가격을 알려줘")
pprint(result.tool_calls)


[{'args': {'date': '2023-12-25', 'symbol': 'TSLA'},
  'id': 'call_7VfzqoCRAmmIA9AO5XbpKNdC',
  'name': 'StockPrice',
  'type': 'tool_call'}]


In [38]:
# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    msg = stock_tool.invoke(tool_call)
    pprint(msg.content)

$TSLA: possibly delisted; no price data found  (1d 2023-12-25 00:00:00 -> 2023-12-26 00:00:00)


('{"symbol": "TSLA", "date": "2025-10-22", "open": 443.45001220703125, "high": '
 '445.5400085449219, "low": 429.0, "close": 438.9700012207031, "volume": '
 '81274900, "comment": "2023-12-25 날짜의 데이터가 없어 최근 거래일인 2025-10-22 날짜의 주식 가격 정보 '
 '조회"}')


---

### [실습]

- BaseTool 상속받아서 도구 정의하는 과정을 살펴 보고, 직접 수정하여 적용해봅니다. 

In [39]:
# BaseTool 상속하여 기간별 주식 데이터 조회 도구 정의

from pydantic import BaseModel, Field

# 입력 스키마 정의
class StockPeriodInput(BaseModel):
    """기간별 주식 데이터 조회 도구의 입력 스키마"""
    symbol: str = Field(description="주식 심볼 (예: AAPL, TSLA)")
    start_date: str = Field(description="시작 날짜 (YYYY-MM-DD 형식)")
    end_date: str = Field(description="종료 날짜 (YYYY-MM-DD 형식)")

# BaseTool 상속하여 도구 구현
class StockPeriodTool(BaseTool):
    name: str = "StockPeriodData"
    description: str = "yfinance를 사용하여 특정 기간 동안의 주식 거래 데이터를 조회합니다."
    args_schema: type[BaseModel] = StockPeriodInput
    return_direct: bool = False

    def _run(
        self,
        symbol: str,
        start_date: str,
        end_date: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> dict:
        """도구를 동기적으로 실행합니다."""
        
        # 날짜 유효성 검사
        if not is_valid_date(start_date):
            raise ToolException(f"잘못된 시작 날짜 형식입니다: {start_date}")
        if not is_valid_date(end_date):
            raise ToolException(f"잘못된 종료 날짜 형식입니다: {end_date}")

        try:
            stock = yf.Ticker(symbol)
            start = datetime.strptime(start_date, "%Y-%m-%d")
            end = datetime.strptime(end_date, "%Y-%m-%d")
            
            # 기간별 주식 가격 정보 조회
            price = stock.history(start=start, end=end)

            # 데이터가 없는 경우
            if price.empty:
                raise ToolException(f"{symbol}의 {start_date}부터 {end_date}까지 주식 데이터를 찾을 수 없습니다.")
            
            # 데이터프레임을 딕셔너리로 변환
            df = price.reset_index()
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
            data = df.to_dict(orient="records")
            
            # 통계 정보 계산
            first_close = data[0].get("Close")
            last_close = data[-1].get("Close")
            change_rate = ((last_close - first_close) / first_close * 100) if first_close else 0
            
            return {
                "symbol": symbol,
                "period": f"{start_date} ~ {end_date}",
                "data_count": len(data),
                "data": data,
                "statistics": {
                    "start_price": first_close,
                    "end_price": last_close,
                    "change_rate": f"{change_rate:.2f}%",
                    "highest": max([d.get("High") for d in data]),
                    "lowest": min([d.get("Low") for d in data]),
                }
            }
                
        except Exception as e:
            raise ToolException(str(e))

    async def _arun(
        self,
        symbol: str,
        start_date: str,
        end_date: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> dict:
        """도구를 비동기적으로 실행합니다."""
        return self._run(symbol, start_date, end_date, run_manager.get_sync() if run_manager else None)

# 도구 생성 및 테스트
stock_period_tool = StockPeriodTool()

# 도구 실행
result = stock_period_tool.invoke({"symbol": "AAPL", "start_date": "2025-01-01", "end_date": "2025-01-10"})
pprint(result)

{'data': [{'Close': 242.98741149902344,
           'Date': '2025-01-02',
           'Dividends': 0.0,
           'High': 248.21884015778812,
           'Low': 240.96459363401945,
           'Open': 248.04942813526918,
           'Stock Splits': 0.0,
           'Volume': 55740700},
          {'Close': 242.49916076660156,
           'Date': '2025-01-03',
           'Dividends': 0.0,
           'High': 243.31625226562878,
           'Low': 241.03435939639962,
           'Open': 242.49916076660156,
           'Stock Splits': 0.0,
           'Volume': 40244100},
          {'Close': 244.1333465576172,
           'Date': '2025-01-06',
           'Dividends': 0.0,
           'High': 246.45510633108967,
           'Low': 242.33971076643533,
           'Open': 243.4457849039297,
           'Stock Splits': 0.0,
           'Volume': 45045600},
          {'Close': 241.3532257080078,
           'Date': '2025-01-07',
           'Dividends': 0.0,
           'High': 244.681407317594,
           'Low': 

---

### 4. **QA 체인 구성** 

- **QA 체인**을 구성하여 질의응답 시스템을 체계화할 수 있음

- **도구 사용**을 통해 사용자가 원하는 정보를 컨텍스트로 활용 가능 

In [40]:
# 도구의 이름 속성 확인 
stock_tool.name

'StockPrice'

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[stock_tool])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def stock_analysis_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        try:
            if tool_call["name"] == "StockPrice":
                tool_result = stock_tool.invoke(tool_call)
            else:
                print(f"알 수 없는 도구 호출: {tool_call['name']}")
                continue

            # tool message 출력
            print("Tool Name: ", tool_call["name"])
            print(tool_result)
            print("-"*200)

            tool_msgs.append(tool_result)
        except Exception as e:
            print(f"{tool_call['name']} 도구 호출 중 에러 발생: {e}")
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "애플의 1월말 주가는 얼마인가요?"
response = stock_analysis_chain.invoke(query)

print(response)



Tool Name:  StockPrice
content='{"symbol": "AAPL", "date": "2025-01-31", "open": 246.31561143302469, "high": 246.31561143302469, "low": 232.61424963135997, "close": 235.16519165039062, "volume": 100959800, "comment": "2025-01-31 날짜의 주식 가격 정보 조회"}' name='StockPrice' tool_call_id='call_rGz6ItcsKZdr7F7HeP22xleo'
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
content='2025년 1월 31일에 애플(AAPL)의 종가는 $235.17였습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 242, 'total_tokens': 266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'f

In [42]:
print(response.content)

2025년 1월 31일에 애플(AAPL)의 종가는 $235.17였습니다.


---

### [심화] **Naver 개발자 API** 를 도구로 사용

#### **환경 설정**

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [43]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다.

    Args:
        query (str): 검색어

    Returns:
        Dict[Dict, int]: 검색 결과와 상태 코드  
    """


    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

---

#### **문제 1**: 도구 실행 체인 구성

1. 도구 설정
    - 도구들(naver_news_search, stock_tool)을 리스트로 구성
    - 도구 이름을 키로 하는 맵(tool_map) 생성

2. 데코레이터 활용
    - @chain 데코레이터로 일반 함수를 체인으로 변환
    - tool_router: 도구 이름에 따라 적절한 도구 선택

3. 체인 구성
    ```python
    tool_chain = (
        llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
        | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
        | tool_router.map()   # 도구 호출 라우팅
    )
    ```

4. 동작 순서
    1. LLM이 상황에 맞는 도구 선택
    2. 해당 도구 실행

In [45]:
from langchain_core.runnables import RunnableLambda

# 1. 도구 설정
tools = [naver_news_search, stock_tool]
tool_map = {tool.name: tool for tool in tools}

print("도구 목록:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")
print()

# 2. 데코레이터 활용 - tool_router
@chain
def tool_router(tool_call):
    """도구 이름에 따라 적절한 도구를 선택하고 실행합니다."""
    tool_name = tool_call.get("name")
    tool = tool_map.get(tool_name)
    
    if tool is None:
        raise ValueError(f"알 수 없는 도구: {tool_name}")
    
    return tool.invoke(tool_call)

# 3. 체인 구성
# LLM에 도구 바인딩
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# tool_chain 구성
tool_chain = (
    llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
    | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
    | tool_router.map()   # 도구 호출 라우팅
)

# 4. 체인 테스트
print("=== 체인 테스트 1: 주식 가격 조회 ===")
result = tool_chain.invoke("애플의 최근 주가를 알려줘")
pprint(result)
print()

print("=== 체인 테스트 2: 뉴스 검색 ===")
result = tool_chain.invoke("애플 관련 뉴스를 검색해줘")
pprint(result)

도구 목록:
  - naver_news_search: 네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다.

    Args:
        query (str): 검색어

    Returns:
        Dict[Dict, int]: 검색 결과와 상태 코드
  - StockPrice: yfinance를 사용하여 특정 날짜의 주식 가격 정보를 조회합니다.

=== 체인 테스트 1: 주식 가격 조회 ===
[ToolMessage(content='{"symbol": "AAPL", "date": "2023-10-30", "open": 167.38054373931297, "high": 169.50968312123874, "low": 167.2319896431063, "close": 168.63821411132812, "volume": 51131000, "comment": "2023-10-30 날짜의 주식 가격 정보 조회"}', name='StockPrice', tool_call_id='call_4faE1CJhPHZKRS8kYxUU6BFB')]

=== 체인 테스트 2: 뉴스 검색 ===
[ToolMessage(content='{"data": {"lastBuildDate": "Thu, 23 Oct 2025 21:41:12 +0900", "total": 1717836, "start": 1, "display": 10, "items": [{"title": "삼성전자, 테슬라 AI5칩 생산 참여", "originallink": "http://www.fnnews.com/news/202510232121072145", "link": "https://n.news.naver.com/mnews/article/014/0005423914?sid=101", "description": "삼성전자는 최근 <b>애플</b>과도 차세대 이미지센서 개발에 나섰다. 미국 오스틴 공장에서 아이폰 등 <b>애플</b> 제품의 전력과 성능을 최적화하는 칩을 공급할 계획이다. 최근 삼성 파운드리의

---

#### **문제 2**: 주식 분석 체인 구성

1. 기본 설정
    ```python
    llm = ChatOpenAI(model="gpt-4.1-mini")
    llm_with_tools = llm.bind_tools([stock_tool, naver_news_search])
    ```

2. 프롬프트 구성
    - 시스템/사용자 메시지 포함
    - 도구 실행 결과를 위한 placeholder 설정

3. 분석 체인 동작
    - 도구 체인으로 주가/뉴스 정보 수집
    - 도구 응답을 메시지 형식으로 변환 
    - LLM으로 최종 분석 응답 생성

4. 데이터 흐름
     - 사용자 입력 -> 도구 실행 -> 결과 포맷팅 -> LLM 분석 -> 최종 응답


In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI
from datetime import datetime

# 1. 기본 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools([stock_tool, naver_news_search])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 2. 프롬프트 구성
prompt = ChatPromptTemplate([
    ("system", f"""당신은 주식 투자 분석을 도와주는 전문 AI 어시스턴트입니다. 
오늘 날짜는 {today}입니다. 
주가 정보와 뉴스 정보를 종합하여 투자 판단에 도움이 되는 분석을 제공하세요.

분석 시 다음 사항을 포함하세요:
- 현재 주가 상황
- 최근 뉴스 동향
- 종합적인 투자 의견
"""),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 3. 분석 체인 동작
@chain
def stock_analysis_chain(user_input: str, config: RunnableConfig):
    """주식 분석 체인 - 주가와 뉴스 정보를 종합하여 분석 제공"""
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        try:
            if tool_call["name"] == "StockPrice":
                tool_result = stock_tool.invoke(tool_call)
            elif tool_call["name"] == "naver_news_search":
                tool_result = naver_news_search.invoke(tool_call)
            else:
                print(f"알 수 없는 도구 호출: {tool_call['name']}")
                continue

            # tool message 출력
            print("Tool Name: ", tool_call["name"])
            print(tool_result.content if hasattr(tool_result, 'content') else tool_result)
            print("-"*200)

            tool_msgs.append(tool_result)
        except Exception as e:
            print(f"{tool_call['name']} 도구 호출 중 에러 발생: {e}")
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 4. 체인 실행
query = "애플 주식에 대해 분석해줘. 최근 주가와 관련 뉴스를 종합해서 알려줘."
response = stock_analysis_chain.invoke(query)

print("\n" + "="*100)
print("=== 최종 분석 결과 ===")
print("="*100)
print(response.content)

Tool Name:  StockPrice
{"symbol": "AAPL", "date": "2025-10-22", "open": 262.7900085449219, "high": 262.8500061035156, "low": 255.42999267578125, "close": 258.45001220703125, "volume": 41141265, "comment": "2025-10-22 날짜의 주식 가격 정보 조회"}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Tool Name:  naver_news_search
{"data": {"lastBuildDate": "Thu, 23 Oct 2025 21:41:15 +0900", "total": 143267, "start": 1, "display": 10, "items": [{"title": "[게임뉴스] 라이온하트, 직원 대상 AI 실무 교육 진행 외 주요 소식", "originallink": "https://www.gamevu.co.kr/news/articleView.html?idxno=52645", "link": "https://www.gamevu.co.kr/news/articleView.html?idxno=52645", "description": "- 아크시스템웍스아시아 <b>주식</b>회사는 오는 11월 27일 '버블보블 슈가 던전' 정식 출시를 앞두고 예약 판매가... 이번 사전예약은 공식 홈페이지와 구글 플레이 스토어, <b>애플</b> 앱스토어에서 진행된다. '전지적 시점: 데몬... ", "pubDate": "Thu, 23 Oct 2025 18:56:00 +0900"}, {"title": "[